## Linear Programming

Contoh Kasus: Perusahaan Batik "Maju Jaya"

Perusahaan batik memproduksi 2 jenis produk:

-    Batik Tulis (x1): Keuntungan Rp 150.000 per unit

-    Batik Cap (x2): Keuntungan Rp 80.000 per unit

Kendala Produksi:

-    Tenaga Kerja: Maksimal 200 jam/bulan

     -   Batik Tulis: 4 jam/unit

     -   Batik Cap: 2 jam/unit

-    Kain: Maksimal 150 meter/bulan

     -   Batik Tulis: 3 meter/unit

     -   Batik Cap: 2 meter/unit

-    Permintaan Pasar:

     -   Batik Tulis maksimal 40 unit/bulan

     -   Batik Cap minimal 20 unit/bulan

Perusahaan ingin memaksimalkan profit dengan berbagai kendala yang ada.

Formulasi Matematis:

Maksimalkan: `Z = 150.000x~1~ + 80.000x2`

Dengan kendala:

-    `4x1 + 2x2 ≤ 200` (tenaga kerja)

-    `3x1 + 2x2` ≤ 150 (kain)

-    `x1 ≤ 40` (permintaan batik tulis)

-    `x2 ≥ 20` (permintaan batik cap)

-    `x1, x2 ≥ 0` (non-negatif)

In [ ]:
# Install library yang diperlukan
!pip install pulp

import pulp
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# =======================
# FORMULASI LINEAR PROGRAMMING
# =======================

# Inisialisasi masalah maksimisasi
model = pulp.LpProblem("Maximize_Profit_Batik", pulp.LpMaximize)

# Variabel keputusan
x1 = pulp.LpVariable('Batik_Tulis', lowBound=0, cat='Continuous')
x2 = pulp.LpVariable('Batik_Cap', lowBound=20, cat='Continuous')  # minimal 20 unit

# Fungsi tujuan
model += 150000*x1 + 80000*x2, "Total_Profit"

# Kendala
model += 4*x1 + 2*x2 <= 200, "Tenaga_Kerja"
model += 3*x1 + 2*x2 <= 150, "Kain"
model += x1 <= 40, "Permintaan_Batik_Tulis"

# Solve model
model.solve()

In [ ]:
# =======================
# HASIL DAN VISUALISASI
# =======================

print("Status:", pulp.LpStatus[model.status])
print("\n=== HASIL OPTIMASI ===")
print(f"Batik Tulis yang diproduksi: {x1.varValue:.0f} unit")
print(f"Batik Cap yang diproduksi: {x2.varValue:.0f} unit")
print(f"Keuntungan Maksimal: Rp {pulp.value(model.objective):,.0f}")

# Hitung penggunaan sumber daya
print("\n=== PENGGUNAAN SUMBER DAYA ===")
print(f"Tenaga kerja terpakai: {4*x1.varValue + 2*x2.varValue:.0f} jam dari 200 jam")
print(f"Kain terpakai: {3*x1.varValue + 2*x2.varValue:.0f} meter dari 150 meter")

In [ ]:
# =======================
# VISUALISASI GRAFIK
# =======================

# Membuat plot
x = np.linspace(0, 50, 400)

# Plot kendala
plt.figure(figsize=(12, 8))

# Kendala tenaga kerja: 4x1 + 2x2 <= 200
y1 = (200 - 4*x) / 2
plt.plot(x, y1, label='4x1 + 2x2 ≤ 200 (Tenaga Kerja)', color='blue')

# Kendala kain: 3x1 + 2x2 <= 150
y2 = (150 - 3*x) / 2
plt.plot(x, y2, label='3x1 + 2x2 ≤ 150 (Kain)', color='red')

# Kendala permintaan batik tulis: x1 <= 40
plt.axvline(x=40, label='x1 ≤ 40 (Permintaan Batik Tulis)', color='green', linestyle='--')

# Kendala permintaan batik cap: x2 >= 20
plt.axhline(y=20, label='x2 ≥ 20 (Permintaan Batik Cap)', color='orange', linestyle='--')

# Area feasible
y_feasible = np.minimum(np.minimum(y1, y2), 100)  # Batas atas untuk plotting
plt.fill_between(x, 20, y_feasible, where=(y_feasible>=20) & (x<=40), alpha=0.3, color='gray', label='Area Feasible')

# Titik optimal
plt.plot(x1.varValue, x2.varValue, 'ro', markersize=10, label=f'Point Optimal ({x1.varValue:.0f}, {x2.varValue:.0f})')

plt.xlim(0, 50)
plt.ylim(0, 100)
plt.xlabel('Batik Tulis (x1)')
plt.ylabel('Batik Cap (x2)')
plt.title('Optimasi Produksi Batik - Daerah Feasible')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# =======================
# ANALISIS SENSITIVITAS
# =======================

print("\n=== ANALISIS SENSITIVITAS ===")
print("Shadow Prices (Dual Values):")
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.pi:.2f}")

print("\nSlack Variables:")
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.slack:.2f}")

#### Shadow Prices (Dual Values) Analysis

Nilai tambahan keuntungan jika constraint dilonggarkan 1 unit

- Tenaga Kerja: -0.00

  -  Menambah 1 jam tenaga kerja tidak akan meningkatkan keuntungan

  -  Sumber daya tenaga kerja sudah berlebih (terlihat dari slack 13.33 jam)

  -  Menambah tenaga kerja justru bisa mengurangi untuk efisiensi

- Kain: 50,000.00

  -  Constraint yang paling kritis!

  -  Setiap penambahan 1 meter kain akan meningkatkan keuntungan sebesar Rp 50,000

  -  Kain adalah bottleneck dalam produksi

-    Rekomendasi:

     -   Prioritaskan penambahan persediaan kain

     -   Jika biaya kain tambahan < Rp 50,000/meter, sangat menguntungkan

     -   Negosiasi dengan supplier untuk tambahan kain

- Permintaan Batik Tulis: -0.00

  -  Meningkatkan batas permintaan Batik Tulis tidak akan menambah keuntungan

  -   Produksi sudah optimal pada level demand saat ini

  -   Fokus pada produk lain atau efisiensi, bukan ekspansi Batik Tulis

#### Slack Variables Analysis

Jumlah resource yang tidak terpakai (sisa)

- Tenaga Kerja: 13.33

  -  Terdapat 13.33 jam tenaga kerja tidak terpakai - underutilized

  -   Realokasi tenaga kerja ke area lain
  -  Pertimbangkan pengurangan jam kerja untuk efisiensi biaya

- Kain: -0.00

   - Tidak ada kain yang tersisa - constraint aktif

   -  Seluruh kain terpakai secara optimal

   -  Penambahan kain akan langsung meningkatkan produksi dan keuntungan

- Permintaan Batik Tulis: 3.33

  - Perusahaan dapat memproduksi 3.33 unit Batik Tulis tambahan

  -  Permintaan pasar belum mencapai kapasitas produksi maksimal

  - Tingkatkan marketing untuk Batik Tulis

  -  Manfaatkan kapasitas produksi yang tersedia

#### Strategic Business Recommendations:
- Priority Actions:

  -  Utamakan Penambahan Kain - ROI tertinggi (Rp 50,000 per meter)

  -  Optimalkan Utilisasi Tenaga Kerja - Kurangi kelebihan 13.33 jam

  -  Tingkatkan Penjualan Batik Tulis - Manfaatkan kapasitas 3.33 unit

- Financial Impact Analysis:

  -  Jika menambah 10 meter kain: +Rp 500,000 keuntungan

  -  Jika mengurangi 10 jam tenaga kerja: Efisiensi biaya tanpa mengurangi profit